In [63]:
#Imports
from thefuzz import process, fuzz
import csv
import snowflake.connector
import pandas as pd
from snowflake.connector.pandas_tools import write_pandas
import matplotlib.pyplot as plt


In [52]:
#Function to read the data from the csv files
def read_csv(file_path):
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        return [row for row in reader]

customers = read_csv('customers.csv')
transactions = read_csv('transactions.csv')

In [53]:
#Function to match customer names
def match_customers(txn_name, customers):
    #Extracting the customer names and ids
    customer_names = [customer['customer_name'] for customer in customers]
    
    #Finding the best match for the txn name in customer name
    best_match, score = process.extractOne(txn_name, customer_names, scorer=fuzz.token_sort_ratio)
    
    #Returning the best match according to the threshold
    if score > 80:  #Threshold can be adjusted, closer to 100 means only very close matches will be accepted
        return next(customer for customer in customers if customer['customer_name'] == best_match)
    return None

#Mergeing the datasets with fuzzy matching
merged_data = []
for transaction in transactions:
    matched_customer = match_customers(transaction['customer_name'], customers)
    if matched_customer:
        merged_data.append({**transaction, **matched_customer})


#Transforming the merged data into pandas df
merged_df = pd.DataFrame(merged_data)

#Displaying 5 rows
print(merged_df.head())  # Displays the first 5 rows


  transaction_id    customer_name amount transaction_date customer_id  \
0            101       John Smith    784       2023-01-31           1   
1            102         Jane Doe    659       2023-02-28           2   
2            103   Micheal Jonson    729       2023-03-31           3   
3            104        Chris Lee    292       2023-04-30           4   
4            105  Katherine Davis    935       2023-05-31           5   

                  email  
0  john.smith@email.com  
1    jane.doe@email.com  
2   m.johnson@email.com  
3       c.lee@email.com  
4     k.davis@email.com  


In [62]:
import snowflake.connector

#Connecting to Snowflake
conn = snowflake.connector.connect(
    user='xxxxx',
    password='xxxxxx',
    account='xxxxxx.east-us-2.azure',
    warehouse='COMPUTE_WH',
    database='FUZZYCHALLENGE',
    schema='PUBLIC',
    role='ACCOUNTADMIN'
)
cursor = conn.cursor()

#SQL script to create a new table
create_table_query = """
CREATE OR REPLACE TABLE fuzzywuzzydata(
    transaction_id integer,
    customer_name string,
    amount integer,
    transaction_date date,
    customer_id integer,
    email string
);
"""

#Executing the script
cursor.execute(create_table_query)

#Saving the file local
merged_df.to_csv('merged_data.csv', index=False)

#Uploading the CSV file to an internal stage
cursor.execute("PUT file://merged_data.csv @%fuzzywuzzydata")

#Loading the data from the stage into the table
cursor.execute("""
COPY INTO fuzzywuzzydata
FROM @%fuzzywuzzydata/merged_data.csv
FILE_FORMAT = (TYPE = CSV, FIELD_OPTIONALLY_ENCLOSED_BY = '"', SKIP_HEADER = 1);
""")

#Making sure it works
print("Data uploaded successfully")

#Closing the cursor and connection
cursor.close()
conn.close()


Data uploaded successfully


Analysis using Jupyter

In [ ]:
#Convert transaction_date to datetime
merged_df['transaction_date'] = pd.to_datetime(merged_df['transaction_date'])

#Converting amount into integer
merged_df['amount'] = pd.to_numeric(merged_df['amount'], downcast='integer')

#Converting customer_id into integer
merged_df['customer_id'] = pd.to_numeric(merged_df['customer_id'], downcast='integer')

#Total sales over time
monthly_sales = merged_df.resample('M', on='transaction_date')['amount'].sum()

#Plotting
plt.figure(figsize=(10, 5))
plt.plot(monthly_sales.index, monthly_sales, marker='o', linestyle='-')
plt.title('Total Sales Over Time')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.grid(True)
plt.show()


In [ ]:
#Aggregating sales by customer_id
customer_sales = merged_df.groupby('customer_id')['amount'].sum().reset_index()

#Plotting
plt.figure(figsize=(10, 5))
plt.hist(customer_sales['amount'], bins=30, edgecolor='k')
plt.title('Customer Sales Distribution')
plt.xlabel('Total Sales per Customer')
plt.ylabel('Number of Customers')
plt.grid(True)
plt.show()


In [ ]:
#Aggregating the number of transactions per customer
customer_purchases = merged_df.groupby('customer_id').size().reset_index(name='num_purchases')

#Plotting
plt.figure(figsize=(10, 5))
plt.bar(customer_purchases['customer_id'].astype(str), customer_purchases['num_purchases'])
plt.title('Number of Transactions per Customer')
plt.xlabel('Customer ID')
plt.ylabel('Number of Transactions')
plt.xticks(rotation=90)
plt.grid(True)
plt.tight_layout()
plt.show()
